# TROISIEME CYCLE

Dans ce troisième cycle, notre but est de prendre les meilleurs modèles Logit et Arbre de décision du deuxième cycle et de leur faire prédire la catégorie d'income des individus du fichier nouvelle_data.csv.

Pour cela, nous devons d'abord mettre en forme le fichier nouvelle_data afin que celui-ci ai la même structure que le fichier data_V2. Nous utilisons la fonction Create_X_nouvelle_data pour appliquer les transformations vues précedemment.Le seul changement réside dans la construction des nouvelles variables qualitatives $\textbf{age}$ et $\textbf{hours-per-week}$ pour lesquelles nous avions précedemment un fichier répertoriant les classes optimisées pour les 48843 individus du fichier d'entraînement. Ici nous avons du créer une nouvelle fonction get_classes qui récupère les classes associées aux deux variables initialement quantitatives. 

Nous effectuons enfin la prédiction en entraînant les modèles de la même manière que les derniers modèles du deuxième cycle. Nous tirerons les prédictions de ces derniers et nous les analyserons rapidement.

In [73]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import tree
import numpy as np

In [74]:
def Create_Train_Test(df):
    """
    Créer les dataframes de test et d'entraînement 
    """
    y = df.pop('income')
    y = pd.get_dummies(y)['>50K']
    X = pd.get_dummies(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0, stratify=y)

    return X_train, X_test, y_train, y_test

In [75]:
def Create_X_nouvelle_data():
    """
    """
    def drop_columns(df):
        """
        Drop les colonnes fnlwgt, educational-nul car inutiles
        """
        df.drop(["fnlwgt", "educational-num"], inplace=True, axis=1)
        return df
    
    def get_classes(df):
        """
        Applique les classes optimisées générées par le fichier optimisation_classes.py
        """

        def classifier(classes, var):
            """
            """
            # Parcourir chaque intervalle dans la liste
            for interval in classes:
                # Extraire l'intervalle sous forme de chaîne
                interval_str = interval[0]

                # Extraire les bornes inférieure et supérieure en enlevant les parenthèses et crochets
                lower_bound, upper_bound = interval_str[1:-1].split(', ')
                
                # Convertir les bornes en float
                lower_bound = float(lower_bound)
                upper_bound = float(upper_bound)

                # Vérifier si l'âge est dans cet intervalle
                if lower_bound < var <= upper_bound:
                    return interval_str
                
        serie_age = pd.read_csv('files/classes_opti/classes_opti_alexander_age.csv')
        serie_hpw = pd.read_csv('files/classes_opti/classes_opti_alexander_hours-per-week.csv')
        
        for serie in [serie_age, serie_hpw]:
            classes = serie.value_counts().index
            df_classes = df[serie.columns[0]].apply(lambda x : classifier(classes, x))
            df.drop(serie.columns[0], inplace=True, axis=1)
            df = pd.concat([df, df_classes], axis=1)

        return df
    
    def regroupement_V2(df):
        """
        """
        # Regroupement des personnes d'ethnies autre que blanc et noir en "Other_race"
        df['race'] = df['race'].replace(
            {'Asian-Pac-Islander|Amer-Indian-Eskimo|Other': 'Other_race'}, regex=True)

        # Regroupement des personnes d'origines différentes en une cartégorie "Other"
        df['native-country'] = df['native-country'].replace(
            {'^(?!United-States$).+': 'Other_country'}, regex=True)

        # Regroupement des personnes mariés mais avec un conjoint dans l'armé avec les personnes mariés mais avec un conjoint absent pour diverse raisons
        df['marital-status'] = df['marital-status'].replace(
            {'Married-AF-spouse|Divorced|Widowed|Separated|Married-spouse-absent': 'Alone'}, regex = True)

        df['education'] = df['education'].replace(
            {'Preschool|1st-4th|5th-6th|7th-8th|9th|10th|11th|12th': 'Low-education',
            'Doctorate|Prof-school': "Graduation",
            "Assoc-acdm|Assoc-voc" : 'Assoc'}, regex=True)

        df['workclass'] = df['workclass'].replace(
            {'Local-gov|State-gov': 'State-Local-gov',
            'Never-worked|Without-pay|\?' : 'No_income_or_unknown'}, regex=True)

        df['occupation'] = df['occupation'].replace(
            {'\?|Priv-house-serv|Other-service|Handlers-cleaners': 'Occupation:Very-Low-income',
            'Farming-fishing|Machine-op-inspct|Adm-clerical': "Occupation:Low-income",
            'Transport-moving|Craft-repair' : 'Occupation:Mid-income',
            'Sales|Armed-Forces|Tech-support|Protective-serv|Protective-serv' :'Occupation:High-income',
            'Prof-specialty|Exec-managerial' : 'Occupation:Very-High-income'}, regex=True)
        
        return df

    df_nouvelle_data = pd.read_csv('files/nouvelle_data.csv')
    df_nouvelle_data = drop_columns(df_nouvelle_data)
    df_nouvelle_data = get_classes(df_nouvelle_data)
    df_nouvelle_data = regroupement_V2(df_nouvelle_data)
    X = pd.get_dummies(df_nouvelle_data)

    return X

In [76]:
def Logistic_Regression_V3(X_train, y_train):
    """
    Crée le Logit et l'entraîne grâce aux datasets d'entraînement
    Avec les hyper-paramètres du GridSearchCV
    """
    model = LogisticRegression(random_state = 42, 
                               C=10,
                               fit_intercept=False,
                               max_iter = 500,
                               tol = 0.0001,
                               solver='lbfgs').fit(X_train, y_train)

    return model

def DecisionTree_V3(X_train, y_train):
    """
    Créer l'arbre de décision et l'entraîne grâce aux datasets d'entraînement
    Un seul hyper-paramètres : max_depth = 7
    """
    model = tree.DecisionTreeClassifier(max_depth=13,
                                        max_leaf_nodes=63,
                                        min_samples_leaf=20, 
                                        min_samples_split=60).fit(X_train, y_train)
    return model


In [77]:
df_V2 = pd.read_csv('files/data/data_V2.csv')
X_train, X_test, y_train, y_test = Create_Train_Test(df_V2)
model_decision_tree = DecisionTree_V3(X_train, y_train)
X_nouvelle_data = Create_X_nouvelle_data()
predict_tree = model_decision_tree.predict(X_nouvelle_data)

In [78]:
df_V2 = pd.read_csv('files/data/data_V2.csv')
X_train, X_test, y_train, y_test = Create_Train_Test(df_V2)
model_logit = Logistic_Regression_V3(X_train, y_train)
X_nouvelle_data = Create_X_nouvelle_data()
predict_logit = model_logit.predict(X_nouvelle_data)

/Users/alexanderlunel/Documents/LILLE/Master/MasterSIAD/M2/Scoring/Etude de cas/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
predict = pd.concat([pd.Series(predict_tree), pd.Series(predict_logit)], axis=1)
predict.columns = ['Tree', 'Logit']

conf_matrix = confusion_matrix(predict['Tree'], predict['Logit'], labels=[True, False])
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Tree >50k', 'Tree <=50k'], columns=['Logit >50k', 'Logit <=50k'])
prc_df_conf_matrix = df_conf_matrix/len(predict)*100
print(df_conf_matrix)
print(prc_df_conf_matrix)
print("Pourcentage de prédictions identiques:", f"{round(prc_df_conf_matrix['Logit >50k']['Tree >50k']+prc_df_conf_matrix['Logit <=50k']['Tree <=50k'], 2)}%")
print("Pourcentage de prédictions différentes", f"{round(prc_df_conf_matrix['Logit <=50k']['Tree >50k']+prc_df_conf_matrix['Logit >50k']['Tree <=50k'], 2)}%")
print("Pourcentage de prédictions >50k Logit", f"{round(df_conf_matrix['Logit >50k'].sum()/len(predict)*100, 2)}%")
print("Pourcentage de prédictions >50k Logit", f"{round(df_conf_matrix.T['Tree >50k'].sum()/len(predict)*100, 2)}%")
print("Nombre de prédictions >50k Logit", df_conf_matrix['Logit >50k'].sum())
print("Nombre de prédictions >50k Tree", df_conf_matrix.T['Tree >50k'].sum())

            Logit >50k  Logit <=50k
Tree >50k         1086         1102
Tree <=50k         345         7148
            Logit >50k  Logit <=50k
Tree >50k    11.217849    11.383122
Tree <=50k    3.563681    73.835348
Pourcentage de prédictions identiques: 85.05%
Pourcentage de prédictions différentes 14.95%
Pourcentage de prédictions >50k Logit 14.78%
Pourcentage de prédictions >50k Logit 22.6%
Nombre de prédictions >50k Logit 1431
Nombre de prédictions >50k Tree 2188
